## Train a model locally or remote

In [15]:
!pip install --upgrade sagemaker==1.60.2

     |████████████████████████████████| 290 kB 39.8 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.60.2-py2.py3-none-any.whl size=392361 sha256=e0b80479a4ef011b58a1b085fa699694488cca0206e8b97fa72f71fe0700505d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/af/35/94/2bb2d2638a2be23d1a1ab9bff1040f96dc76ef0086dd52da77
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 1.58.2
    Uninstalling sagemaker-1.58.2:
      Successfully uninstalled sagemaker-1.58.2
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [16]:
from sklearn import datasets, svm
digits = datasets.load_digits()
clf = svm.SVC(gamma=0.001, C=100.,probability=True)
clf.fit(digits.data[:-1], digits.target[:-1])
clf.predict(digits.data[-1:])

array([8])

In [17]:
!pip show scikit-learn

Name: scikit-learn
Version: 0.22.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages
Requires: numpy, joblib, scipy
Required-by: sklearn, scikit-optimize, mleap, lightgbm, autogluon


### Save model file

In [18]:
from joblib import dump
dump(clf, 'model.joblib')

['model.joblib']

## Step 1 : Write a model transform script

#### Make sure you have a ...

- "load_model" function
    - input args are model path
    - returns loaded model object
    - model name is the same as what you saved the model file as (see above step)
<br><br>
- "predict" function
    - input args are the loaded model object and a payload
    - returns the result of model.predict
    - make sure you format it as a single (or multiple) string return inside a list for real time (for mini batch)
    - from a client, a list  or string or np.array that is sent for prediction is interpreted as bytes. Do what you have to for converting back to list or string or np.array
    - return the error for debugging


In [19]:
%%writefile modelscript_sklearn.py
import sklearn
from joblib import load
import numpy as np
import os

#Return loaded model
def load_model(modelpath):
    print(modelpath)
    clf = load(os.path.join(modelpath,'model.joblib'))
    print("loaded")
    return clf

# return prediction based on loaded model (from the step above) and an input payload
def predict(model, payload):
    try:
        # locally, payload may come in as an np.ndarray
        if type(payload)==np.ndarray:
            out = [str(model.predict(np.frombuffer(payload).reshape((1,64))))]
        # in remote / container based deployment, payload comes in as a stream of bytes
        else:
            out = [str(model.predict(np.frombuffer(payload[0]['body']).reshape((1,64))))]
    except Exception as e:
        out = [type(payload),str(e)] #useful for debugging!
    
    return out

Overwriting modelscript_sklearn.py


## Does this work locally? (not "_in a container locally_", but _actually_ in local)

In [20]:
from modelscript_sklearn import *
model = load_model('.')

.
loaded


In [21]:
predict(model,digits.data[-1:])[0]

'[8]'

### ok great! Now let's install ezsmdeploy

In [23]:
!pip install ezsmdeploy==0.2.8

Processing /home/ec2-user/.cache/pip/wheels/08/75/84/62b3bde8befd85b487082ff95834d0357efbd5ae73408b2ed7/sagemaker-1.58.2-py2.py3-none-any.whl
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 1.60.2
    Uninstalling sagemaker-1.60.2:
      Successfully uninstalled sagemaker-1.60.2
  Attempting uninstall: ezsmdeploy
    Found existing installation: ezsmdeploy 0.2.6
    Uninstalling ezsmdeploy-0.2.6:
      Successfully uninstalled ezsmdeploy-0.2.6
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [27]:
!pip install --upgrade sagemaker==1.60.2

Processing /home/ec2-user/.cache/pip/wheels/af/35/94/2bb2d2638a2be23d1a1ab9bff1040f96dc76ef0086dd52da77/sagemaker-1.60.2-py2.py3-none-any.whl
ERROR: ezsmdeploy 0.2.8 has requirement sagemaker==1.58.2, but you'll have sagemaker 1.60.2 which is incompatible.
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 1.58.2
    Uninstalling sagemaker-1.58.2:
      Successfully uninstalled sagemaker-1.58.2
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [28]:
import ezsmdeploy

#### If you have been running other inference containers in local mode, stop existing containers to avoid conflict

In [29]:
!docker container stop $(docker container ls -aq) >/dev/null

tmpwku_48_g_algo-1-xog72_1 exited with code 137
Aborting on container exit...


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 618, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 677, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 623, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpwku_48_g/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 137



## Deploy locally

In [30]:
ez = ezsmdeploy.Deploy(model = ['model.joblib'], # if you intend to add models later, pass model as list, otherwise str
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.22.1','numpy','joblib'], #or pass in the path to requirements.txt
                  instance_type = 'local',
                  autoscale = True,
                  #framework = 'sklearn', # not required if you provide requirements
                  wait = True)

0:00:00.176737 | compressed model(s)
0:00:00.270110 | uploaded model tarball(s) ; check returned modelpath
0:00:00.270951 | added requirements file
0:00:00.272734 | added source file
0:00:00.274217 | added Dockerfile
0:00:00.275844 | added model_handler and docker utils
0:00:00.275934 | building docker container
0:00:32.966161 | built docker container
0:00:33.262990 | created model(s). Now deploying on local
∙∙∙ Attaching to tmpxsoj8tbp_algo-1-egc7w_1
algo-1-egc7w_1  | Warning: Calling MMS with mxnet-model-server. Please move to multi-model-server.
∙∙● algo-1-egc7w_1  | 2020-05-29 22:25:34,636 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
algo-1-egc7w_1  | MMS Home: /usr/local/lib/python3.5/dist-packages
algo-1-egc7w_1  | Current directory: /
algo-1-egc7w_1  | Temp directory: /tmp
algo-1-egc7w_1  | Number of GPUs: 0
algo-1-egc7w_1  | Number of CPUs: 32
algo-1-egc7w_1  | Max heap size: 27305 M
algo-1-egc7w_1  | Python executable: /usr/bin/python3
algo-1-egc7w_1  | Config file: /etc/s

## Test containerized version locally

In [31]:
out = ez.predictor.predict(digits.data[-1:].tobytes()).decode()
out

algo-1-egc7w_1  | 2020-05-29 22:28:32,384 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 2
algo-1-egc7w_1  | 2020-05-29 22:28:32,384 [INFO ] W-9000-model ACCESS_LOG - /172.28.0.1:53030 "POST /invocations HTTP/1.1" 200 5


'[8]'

In [32]:
!docker container stop $(docker container ls -aq) >/dev/null

tmpxsoj8tbp_algo-1-egc7w_1 exited with code 137
Aborting on container exit...


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 618, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 677, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 623, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpxsoj8tbp/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 137



## Deploy on SageMaker

In [33]:
ezonsm = ezsmdeploy.Deploy(model = ['model.joblib','model.joblib'], # example of multimodel endpoint. 
                  script = 'modelscript_sklearn.py',
                  requirements = ['pyarrow','scikit-learn==0.22.1','numpy','joblib'])

0:00:00.350628 | compressed model(s)
0:00:00.618988 | uploaded model tarball(s) ; check returned modelpath
0:00:00.620021 | added requirements file
0:00:00.622249 | added source file
0:00:00.623637 | added Dockerfile
0:00:00.625397 | added model_handler and docker utils
0:00:00.625486 | building docker container
0:00:33.064223 | built docker container
0:00:33.553717 | created model(s). Now deploying on ml.m5.xlarge
0:08:06.029243 | deployed model
0:08:06.030004 | estimated cost is $0.3 per hour
0:08:06.030133 | Done! ✔ 


In [38]:
!./src/build-docker.sh test

Building container ezsmdeploy-image-test
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
sha256:1e90a329b3b6b8bdd322f7bdf024703582af5f631baaf9dc0b131b75e6c39ea9
The push refers to repository [497456752804.dkr.ecr.us-east-1.amazonaws.com/ezsmdeploy-image-test]

082b50ee: Preparing 
3fc4f971: Preparing 
b9920ba1: Preparing 
c6baf017: Preparing 
3f2c9c7e: Preparing 
976f0b79: Preparing 
ada55966: Preparing 
188ddb3a: Preparing 
c8dacd6e: Preparing 
4b9b657a: Preparing 
e85730ff: Preparing 
e4aaa54c: Preparing 
33376c2f: Preparing 
8ea09024: Preparing 
e85730ff: Pushed     504MB/494.8MBPushing  221.4MB/494.8MBPushing  392.6MB/494.8MBPushing    433MB/494.8MBlatest: digest: sha256:2918580c1c04eb159bebc25b8eeb57100453cd9e4916fe97e

In [34]:
!pip show sagemaker

Name: sagemaker
Version: 1.60.2
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages
Requires: importlib-metadata, protobuf, numpy, smdebug-rulesconfig, boto3, packaging, scipy, protobuf3-to-dict
Required-by: ezsmdeploy


In [35]:
out = ezonsm.predictor.predict(digits.data[-1:].tobytes(),target_model='model1.tar.gz').decode() 
out

'[8]'

In [ ]:
ezonsm.test(input_data=digits.data[-1:].tobytes(), target_model='model1.tar.gz',usercount=20,hatchrate=10,timeoutsecs=10)

In [ ]:
import pandas as pd

pd.read_csv('src/locuststats_stats.csv')

In [ ]:
ezonsm.predictor.delete_endpoint()